# Serial Value Iteration

We first load PLite into the current process.

In [2]:
push!(LOAD_PATH, "../src")
using PLite

## Problem definition

For this example, we define a simple 1-D gridworld type problem where the goal of the agent is to move and stop at the center of the grid. 

The idea of this section is to define the problem mathematically without fussing about how we might choose to solve it later (e.g., use a discretized representation). In general, given the wide variety of MDP solvers available, it's important not to restrict our problem representation by what solver we might eventually choose. Rather, we want to choose the best solver after understanding our problem (e.g., find some problem structure that we can exploit).

### MDP initialization

We first define some constants and initialize the empty `MDP` object.

In [3]:
# constants
const MinX = 0
const MaxX = 100
const StepX = 20

# mdp definition
mdp = MDP()

PLite.MDP(Dict{AbstractString,PLite.LazyVar}(),Dict{AbstractString,PLite.LazyVar}(),PLite.LazyFunc(true,ASCIIString[],##emptyfunc#7460),PLite.LazyFunc(true,ASCIIString[],##emptyfunc#7459))

### State and action spaces definition

We define the state and action spaces using a factored representation. If we can't factor the representation, we can simply define the space using a single discrete or continuous variable. 

For each state and action variable, we either define a continuous or discrete variable. Note that we use the state variables' "natural representation" and avoid discretizing `x` even though for value iteration we would eventually have to. 

In [4]:
# state space
statevariable!(mdp, "x", MinX, MaxX)  # continuous
statevariable!(mdp, "goal", ["no", "yes"])  # discrete

# action space
actionvariable!(mdp, "move", ["W", "E", "stop"])  # discrete

PLite.ValuesVar("move",ASCIIString["W","E","stop"])

### Transition and reward functions definition

To define either the transition or reward function, we pass in the `MDP` object, the transition function itself, and an ordered set of the transition function's argument names. 

In the case of the transition function, we can define it using either the $T(s,a)$ or $T(s,a,s')$ format. Our example here uses the former way. In the case of the reward function, we define it using the $R(s,a)$ format.

In [5]:
transition!(mdp,
  ["x", "goal", "move", "x", "goal"],  # note |xp| is an "x" variable
                                       # note (s,a,s') order
  function mytransition(
      x::Float64,
      goal::AbstractString,
      move::AbstractString,
      xp::Float64,
      goalp::AbstractString)

    function internaltransition(x::Float64, goal::AbstractString, move::AbstractString)
      function isgoal(x::Float64)
        if abs(x - MaxX / 2) < StepX
          return "yes"
        else
          return "no"
        end
      end

      if isgoal(x) == "yes" && goal == "yes"
        return [([x, isgoal(x)], 1.0)]
      end

      if move == "E"
        if x >= MaxX
          return [
            ([x, isgoal(x)], 0.9),
            ([x - StepX, isgoal(x - StepX)], 0.1)]
        elseif x <= MinX
          return [
            ([x, isgoal(x)], 0.2),
            ([x + StepX, isgoal(x + StepX)], 0.8)]
        else
          return [
            ([x, isgoal(x)], 0.1),
            ([x - StepX, isgoal(x - StepX)], 0.1),
            ([x + StepX, isgoal(x + StepX)], 0.8)]
        end
      elseif move == "W"
        if x >= MaxX
          return [
            ([x, isgoal(x)], 0.1),
            ([x - StepX, isgoal(x - StepX)], 0.9)]
        elseif x <= MinX
          return [
          ([x, isgoal(x)], 0.9),
          ([x + StepX, isgoal(x + StepX)], 0.1)]
        else
          return [
            ([x, isgoal(x)], 0.1),
            ([x - StepX, isgoal(x - StepX)], 0.8),
            ([x + StepX, isgoal(x + StepX)], 0.1)]
        end
      elseif move == "stop"
        return [([x, isgoal(x)], 1.0)]
      end
    end

    statepprobs = internaltransition(x, goal, move)
    for statepprob in statepprobs
      if xp == statepprob[1][1] && goalp == statepprob[1][2]
        return statepprob[2]
      end
    end
    return 0

  end
)

  likely near In[5]:1
  likely near In[5]:1
  likely near In[5]:1


PLite.LazyFunc(false,ASCIIString["x","goal","move","x","goal"],mytransition)

## Infinite horizon MDP

For an infinite horizon problem with discount $\gamma$, it can be proven that the value of an optimal policy satisfies the *Bellman equation*
$$U^{\star}(s) = \max_{a}\left( R\left(s,a\right) + \gamma\sum_{s'}T\left(s'\mid s, a\right) U^{\star}\left(s'\right) \right)\text{.}$$
For the convergence proof, see http://web.stanford.edu/class/ee266/lectures/dpproof.pdf.

## Finite horizon MDP

In value iteration, we compute optimal value function $U_{n}$ associated with a finite horizon of $n$ and no discounting. If $n=0$, then $U_{0}(s)=0$ for all $s$. We can compute $U_{n}$ recursively from this base case
$$U_{n}(s) = \max_{a}\left( R\left(s,a\right) + \sum_{s'}T\left(s'\mid s, a\right) U_{n-1}\left(s'\right) \right)\text{.}$$

## Policy extraction

The optimal value function $U^{\star}$ appears on both sides of the equation. Value iteration approximates $U^{\star}$ by iteratively updating the estimate of $U^{\star}$ using the above equation. Once we know $U^{\star}$, we can extract an optimal policy via
$$\pi\left(s\right) \leftarrow \underset{a} {\mathrm{argmax}} \left( R\left(s,a\right) + \gamma\sum_{s'}T\left(s'\mid s, a\right) U^{\star}\left(s'\right) \right)\text{.}$$